In [49]:
import requests


In [50]:
standings_url = "https://fbref.com/en/comps/70/Saudi-Professional-League-Stats"


In [51]:
data = requests.get(standings_url)


In [52]:
from bs4 import BeautifulSoup


In [53]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [54]:
team_urls = [f"https://fbref.com{l}" for l in links]


In [55]:
data = requests.get(team_urls[0])


In [56]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [57]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [58]:
data = requests.get(f"https://fbref.com{links[0]}")


In [59]:
shooting = pd.read_html(data.text, match="Shooting")[0]


In [60]:
shooting.head()


For Al-Hilal                                                              \
          Date   Time        Comp        Round  Day Venue Result   GF   GA   
0   2023-08-14  18:00  Pro League  Matchweek 1  Mon  Away      W  3.0  1.0   
1   2023-08-19  21:00  Pro League  Matchweek 2  Sat  Home      D  1.0  1.0   
2   2023-08-24  21:00  Pro League  Matchweek 3  Thu  Away      W  4.0  0.0   
3   2023-08-28  21:00  Pro League  Matchweek 4  Mon  Home      W  2.0  0.0   
4   2023-09-01  21:00  Pro League  Matchweek 5  Fri  Away      W  4.0  3.0   

              Standard                                          \
     Opponent      Gls  Sh SoT  SoT%  G/Sh G/SoT Dist PK PKatt   
0        Abha        3  26   8  30.8  0.12  0.38  NaN  0     0   
1    Al-Fayha        1  20   8  40.0  0.05  0.13  NaN  0     0   
2     Al-Raed        4  20  10  50.0  0.15  0.30  NaN  1     1   
3  Al-Ettifaq        2  21   6  28.6  0.10  0.33  NaN  0     0   
4  Al-Ittihad        4  14   7  50.0  0.21  0.43  NaN  1     1   

  Unnamed: 19_level_0  
         Match Report  
0        Match Report  
1        Match Report  
2        Match Report  
3        Match Report  
4        Match Report

In [61]:
shooting.columns = shooting.columns.droplevel()


In [62]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist","PK", "PKatt"]], on="Date")


In [63]:
team_data.head()


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Captain,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,PK,PKatt
0,2023-08-14,18:00,Pro League,Matchweek 1,Mon,Away,W,3.0,1.0,Abha,...,Abdullah Al-Mayouf,4-2-3-1,Radu Petrescu,Match Report,NaN,26,8,NaN,0,0
1,2023-08-19,21:00,Pro League,Matchweek 2,Sat,Home,D,1.0,1.0,Al-Fayha,...,Abdullah Al-Mayouf,4-2-3-1,Yael Falcón,Match Report,NaN,20,8,NaN,0,0
2,2023-08-24,21:00,Pro League,Matchweek 3,Thu,Away,W,4.0,0.0,Al-Raed,...,Salem Al-Dawsari,4-2-3-1,César Arturo Ramos,Match Report,NaN,20,10,NaN,1,1
3,2023-08-28,21:00,Pro League,Matchweek 4,Mon,Home,W,2.0,0.0,Al-Ettifaq,...,Salem Al-Dawsari,4-2-3-1,Ramon Abatti,Match Report,NaN,21,6,NaN,0,0
4,2023-09-01,21:00,Pro League,Matchweek 5,Fri,Away,W,4.0,3.0,Al-Ittihad,...,Salem Al-Dawsari,4-2-3-1,Mario Escobar,Match Report,NaN,14,7,NaN,1,1


In [64]:
years = list(range(2024, 2022, -1))
all_matches = []

In [65]:
standings_url = "https://fbref.com/en/comps/70/Saudi-Professional-League-Stats"


In [66]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist",  "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(4)

In [67]:
len(all_matches)


34

In [68]:
all_matches

[          Date   Time        Comp         Round  Day Venue Result   GF   GA  \
 0   2023-08-14  18:00  Pro League   Matchweek 1  Mon  Away      W  3.0  1.0   
 1   2023-08-19  21:00  Pro League   Matchweek 2  Sat  Home      D  1.0  1.0   
 2   2023-08-24  21:00  Pro League   Matchweek 3  Thu  Away      W  4.0  0.0   
 3   2023-08-28  21:00  Pro League   Matchweek 4  Mon  Home      W  2.0  0.0   
 4   2023-09-01  21:00  Pro League   Matchweek 5  Fri  Away      W  4.0  3.0   
 5   2023-09-15  21:00  Pro League   Matchweek 6  Fri  Home      W  6.0  1.0   
 6   2023-09-21  18:00  Pro League   Matchweek 7  Thu  Away      D  1.0  1.0   
 7   2023-09-29  21:00  Pro League   Matchweek 8  Fri  Home      W  2.0  0.0   
 8   2023-10-07  18:00  Pro League   Matchweek 9  Sat  Away      W  3.0  0.0   
 9   2023-10-20  21:00  Pro League  Matchweek 10  Fri  Home      W  1.0  0.0   
 10  2023-10-27  21:00  Pro League  Matchweek 11  Fri  Home      W  3.0  1.0   
 11  2023-11-03  18:00  Pro League  Matc

In [70]:
match_df = pd.concat(all_matches)


In [71]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Referee,Match Report,Notes,Sh,SoT,Dist,PK,PKatt,Season,Team
0,2023-08-14,18:00,Pro League,Matchweek 1,Mon,Away,W,3.0,1.0,Abha,...,Radu Petrescu,Match Report,NaN,26,8,NaN,0,0,2024,Al Hilal
1,2023-08-19,21:00,Pro League,Matchweek 2,Sat,Home,D,1.0,1.0,Al-Fayha,...,Yael Falcón,Match Report,NaN,20,8,NaN,0,0,2024,Al Hilal
2,2023-08-24,21:00,Pro League,Matchweek 3,Thu,Away,W,4.0,0.0,Al-Raed,...,César Arturo Ramos,Match Report,NaN,20,10,NaN,1,1,2024,Al Hilal
3,2023-08-28,21:00,Pro League,Matchweek 4,Mon,Home,W,2.0,0.0,Al-Ettifaq,...,Ramon Abatti,Match Report,NaN,21,6,NaN,0,0,2024,Al Hilal
4,2023-09-01,21:00,Pro League,Matchweek 5,Fri,Away,W,4.0,3.0,Al-Ittihad,...,Mario Escobar,Match Report,NaN,14,7,NaN,1,1,2024,Al Hilal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,2023-05-10,19:00,Pro League,Matchweek 26,Wed,Home,W,4.0,3.0,Al-Ta'ee,...,Mohammed Khaled Al Hoish,Match Report,NaN,20,10,NaN,1,1,2023,Al Batin
26,2023-05-15,19:00,Pro League,Matchweek 27,Mon,Home,D,0.0,0.0,Al-Fayha,...,Sultan Sanat Al Harbi,Match Report,NaN,6,0,NaN,0,0,2023,Al Batin
27,2023-05-23,21:30,Pro League,Matchweek 28,Tue,Away,L,0.0,1.0,Al-Ittihad,...,Tamás Bognár,Match Report,NaN,3,0,NaN,0,0,2023,Al Batin
28,2023-05-27,21:00,Pro League,Matchweek 29,Sat,Home,D,2.0,2.0,Al-Adalah,...,Faisal Suleiman Al Balawi,Match Report,NaN,6,2,NaN,0,0,2023,Al Batin


In [72]:
match_df.columns = [c.lower() for c in match_df.columns]


In [74]:
match_df.to_csv("matches1.csv")

In [81]:
from google.colab import files

# Assuming your DataFrame is named match_df
match_df.to_csv("matches1.csv", index=False)

# Download the CSV file
files.download("matches1.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [79]:
import pandas as pd
matches = pd.read_csv("matches.csv")


In [80]:
matches.head()


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,pk,pkatt,season,team
0,2023-08-14,18:00,Pro League,Matchweek 1,Mon,Away,W,3.0,1.0,Abha,...,Radu Petrescu,Match Report,NaN,26,8,NaN,0,0,2024,Al Hilal
1,2023-08-19,21:00,Pro League,Matchweek 2,Sat,Home,D,1.0,1.0,Al-Fayha,...,Yael Falcón,Match Report,NaN,20,8,NaN,0,0,2024,Al Hilal
2,2023-08-24,21:00,Pro League,Matchweek 3,Thu,Away,W,4.0,0.0,Al-Raed,...,César Arturo Ramos,Match Report,NaN,20,10,NaN,1,1,2024,Al Hilal
3,2023-08-28,21:00,Pro League,Matchweek 4,Mon,Home,W,2.0,0.0,Al-Ettifaq,...,Ramon Abatti,Match Report,NaN,21,6,NaN,0,0,2024,Al Hilal
4,2023-09-01,21:00,Pro League,Matchweek 5,Fri,Away,W,4.0,3.0,Al-Ittihad,...,Mario Escobar,Match Report,NaN,14,7,NaN,1,1,2024,Al Hilal


In [82]:
matches.shape


(1056, 24)

In [83]:
# 1 seasons * 18 squads  * 34 matches / 1 seansons 16 squads 30 match
34 *18 + 16*30

1092

In [84]:
matches["team"].value_counts()


team
Al Hilal         62
Al Shabab        62
Abha             62
Al Taee          62
Al Wehda         62
Al Raed          62
Al Nassr         62
Damac            62
Al Khaleej       62
Al Fateh         62
Al Fayha         62
Al Ettifaq       62
Al Ittihad       62
Al Taawoun       62
Al Riyadh SC     32
Al Okhdood SC    32
Al Ahli          32
Al Hazem         32
Al Adalah        30
Al Batin         30
Name: count, dtype: int64

In [85]:
matches["date"].value_counts()


date
2023-05-31    16
2023-05-27    16
2023-09-30    10
2022-12-26    10
2023-08-29    10
              ..
2023-01-15     2
2023-01-10     2
2024-05-06     2
2022-09-04     2
2023-02-14     2
Name: count, Length: 186, dtype: int64

In [86]:
matches["round"].value_counts()

round
Matchweek 1     34
Matchweek 2     34
Matchweek 28    34
Matchweek 30    34
Matchweek 29    34
Matchweek 27    34
Matchweek 26    34
Matchweek 25    34
Matchweek 24    34
Matchweek 23    34
Matchweek 22    34
Matchweek 21    34
Matchweek 20    34
Matchweek 19    34
Matchweek 18    34
Matchweek 17    34
Matchweek 16    34
Matchweek 15    34
Matchweek 14    34
Matchweek 13    34
Matchweek 12    34
Matchweek 11    34
Matchweek 10    34
Matchweek 9     34
Matchweek 8     34
Matchweek 7     34
Matchweek 6     34
Matchweek 5     34
Matchweek 4     34
Matchweek 3     34
Matchweek 31    18
Matchweek 32    18
Name: count, dtype: int64

In [102]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh                int64
sot               int64
dist            float64
pk                int64
pkatt             int64
season            int64
team             object
venue_code         int8
opp_code           int8
team_code          int8
hour              int64
target            int64
dtype: object

In [108]:
matches["date"] = pd.to_datetime(matches["date"])


In [109]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes


In [110]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes





In [111]:
matches["day_code"] = matches["date"].dt.dayofweek


In [112]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["target"] = (matches["result"] == "W").astype("int")



In [113]:
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pk,pkatt,season,team,venue_code,opp_code,team_code,hour,target,day_code
0,2023-08-14,18:00,Pro League,Matchweek 1,Mon,Away,W,3.0,1.0,Abha,...,0,0,2024,Al Hilal,0,0,8,18,1,0
1,2023-08-19,21:00,Pro League,Matchweek 2,Sat,Home,D,1.0,1.0,Al-Fayha,...,0,0,2024,Al Hilal,1,7,8,21,0,5
2,2023-08-24,21:00,Pro League,Matchweek 3,Thu,Away,W,4.0,0.0,Al-Raed,...,1,1,2024,Al Hilal,0,13,8,21,1,3
3,2023-08-28,21:00,Pro League,Matchweek 4,Mon,Home,W,2.0,0.0,Al-Ettifaq,...,0,0,2024,Al Hilal,1,5,8,21,1,0
4,2023-09-01,21:00,Pro League,Matchweek 5,Fri,Away,W,4.0,3.0,Al-Ittihad,...,1,1,2024,Al Hilal,0,10,8,21,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,2023-05-10,19:00,Pro League,Matchweek 26,Wed,Home,W,4.0,3.0,Al-Ta'ee,...,1,1,2023,Al Batin,1,16,3,19,1,2
1052,2023-05-15,19:00,Pro League,Matchweek 27,Mon,Home,D,0.0,0.0,Al-Fayha,...,0,0,2023,Al Batin,1,7,3,19,0,0
1053,2023-05-23,21:30,Pro League,Matchweek 28,Tue,Away,L,0.0,1.0,Al-Ittihad,...,0,0,2023,Al Batin,0,10,3,21,0,1
1054,2023-05-27,21:00,Pro League,Matchweek 29,Sat,Home,D,2.0,2.0,Al-Adalah,...,0,0,2023,Al Batin,1,2,3,21,0,5


In [114]:
opp_codes_by_team = matches.groupby("opponent")["opp_code"].unique()

# Affichage des codes pour chaque équipe
for team, codes in opp_codes_by_team.items():
    print(f"Opponent: {team}, Opponent Codes: {', '.join(map(str, codes))}")

Opponent: Abha, Opponent Codes: 0
Opponent: Al Okhdood SC, Opponent Codes: 1
Opponent: Al-Adalah, Opponent Codes: 2
Opponent: Al-Ahli, Opponent Codes: 3
Opponent: Al-Batin, Opponent Codes: 4
Opponent: Al-Ettifaq, Opponent Codes: 5
Opponent: Al-Fateh, Opponent Codes: 6
Opponent: Al-Fayha, Opponent Codes: 7
Opponent: Al-Hazem, Opponent Codes: 8
Opponent: Al-Hilal, Opponent Codes: 9
Opponent: Al-Ittihad, Opponent Codes: 10
Opponent: Al-Khaleej, Opponent Codes: 11
Opponent: Al-Nassr, Opponent Codes: 12
Opponent: Al-Raed, Opponent Codes: 13
Opponent: Al-Riyadh SC, Opponent Codes: 14
Opponent: Al-Shabab, Opponent Codes: 15
Opponent: Al-Ta'ee, Opponent Codes: 16
Opponent: Al-Taawoun, Opponent Codes: 17
Opponent: Al-Wehda, Opponent Codes: 18
Opponent: Damac, Opponent Codes: 19


In [119]:
from sklearn.ensemble import RandomForestClassifier


In [164]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)


In [165]:
predictors = ["venue_code", "opp_code", "hour"]


In [166]:
train = matches[matches["date"] < '2024-2-01']
test =  matches[matches["date"] < '2024-2-01']

In [167]:
rf.fit(train[predictors], train["target"])


RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [168]:
preds = rf.predict(test[predictors])


In [169]:
from sklearn.metrics import accuracy_score
error = accuracy_score(test["target"], preds)


In [170]:
error

0.7439024390243902

In [171]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))


In [172]:
from sklearn.metrics import precision_score

precision_score(test["target"], preds)

0.7394957983193278

In [225]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)

In [226]:
error = precision_score(test["target"], preds)

In [227]:
grouped_matches = matches.groupby("team")


In [228]:
group = grouped_matches.get_group("Al Nassr").sort_values("date")

In [229]:
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pk,pkatt,season,team,venue_code,opp_code,team_code,hour,target,day_code
606,2022-08-27,21:00,Pro League,Matchweek 1,Sat,Home,W,1.0,0.0,Al-Wehda,...,0,0,2023,Al Nassr,1,18,11,21,1,5
607,2022-09-03,19:00,Pro League,Matchweek 2,Sat,Away,L,0.0,1.0,Al-Taawoun,...,0,0,2023,Al Nassr,0,17,11,19,0,5
608,2022-09-10,21:00,Pro League,Matchweek 3,Sat,Home,W,2.0,1.0,Damac,...,0,0,2023,Al Nassr,1,19,11,21,1,5
609,2022-09-15,18:40,Pro League,Matchweek 4,Thu,Away,W,4.0,0.0,Al-Batin,...,0,0,2023,Al Nassr,0,4,11,18,1,3
610,2022-10-02,21:00,Pro League,Matchweek 5,Sun,Home,D,0.0,0.0,Al-Ittihad,...,0,0,2023,Al Nassr,1,10,11,21,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2024-04-19,18:00,Pro League,Matchweek 28,Fri,Home,W,3.0,1.0,Al-Fayha,...,0,1,2024,Al Nassr,1,7,11,18,1,4
60,2024-04-27,21:00,Pro League,Matchweek 29,Sat,Away,W,1.0,0.0,Al-Khaleej,...,0,0,2024,Al Nassr,0,11,11,21,1,5
61,2024-05-04,21:00,Pro League,Matchweek 30,Sat,Home,W,6.0,0.0,Al-Wehda,...,0,0,2024,Al Nassr,1,18,11,21,1,5
62,2024-05-09,21:00,Pro League,Matchweek 31,Thu,Away,W,3.0,2.0,Al Okhdood SC,...,0,0,2024,Al Nassr,0,1,11,21,1,3


In [231]:
combined = combined.merge(group[["date", "team", "opponent", "result"]], left_index=True, right_index=True)


In [232]:
combined

,actual,predicted,date,team,opponent,result
32,0,0,2023-08-14,Al Nassr,Al-Ettifaq,L
33,0,0,2023-08-18,Al Nassr,Al-Taawoun,L
34,1,0,2023-08-25,Al Nassr,Al-Fateh,W
35,1,1,2023-08-29,Al Nassr,Al-Shabab,W
36,1,1,2023-09-02,Al Nassr,Al-Hazem,W
37,1,1,2023-09-16,Al Nassr,Al-Raed,W
38,1,0,2023-09-22,Al Nassr,Al-Ahli,W
39,1,0,2023-09-29,Al Nassr,Al-Ta'ee,W
40,0,1,2023-10-06,Al Nassr,Abha,D
41,1,1,2023-10-21,Al Nassr,Damac,W
